In [1]:
import numpy as np
from scipy.spatial.distance import cdist

In [2]:
np.random.seed(42)

In [9]:
x = (np.random.uniform(0,15,size=[1500,300])).astype(np.float64)
y = (np.random.uniform(100,459,size=[6000,300])).astype(np.float64)

In [10]:
x

array([[12.91631763,  0.52605185, 10.48249454, ...,  3.4531903 ,
         7.70545674,  2.41203389],
       [ 2.25816089,  5.12286375,  1.30117444, ...,  9.85682736,
         7.83286379, 10.1904424 ],
       [ 7.54211774,  6.52450717, 11.85271016, ...,  0.9868032 ,
         9.97605198, 10.61735318],
       ...,
       [ 5.50157401,  3.62038599,  3.71020266, ...,  5.57154287,
         7.03493786, 10.24818671],
       [14.07167628,  3.94016593,  0.42737879, ...,  8.94926331,
         8.50775393,  5.09276834],
       [ 4.76718858, 13.20636999,  0.05059372, ...,  5.77950858,
        10.53735267,  0.31888505]])

In [11]:
y

array([[226.56335938, 398.72389027, 457.61277215, ..., 433.78638312,
        295.8689927 , 333.95808102],
       [218.7458713 , 329.23402277, 400.06692678, ..., 374.87539959,
        160.30566516, 300.34698842],
       [259.69592749, 144.41448024, 180.4793175 , ..., 299.00815208,
        414.89324398, 102.264541  ],
       ...,
       [364.51655469, 320.42866137, 326.84563975, ..., 258.10939632,
        201.05118   , 338.98353221],
       [434.52912907, 451.05399878, 140.53288559, ..., 290.37744899,
        297.70729251, 395.48584835],
       [435.35389602, 367.03859798, 109.62792566, ..., 383.51105903,
        189.66069908, 315.03980737]])

In [12]:
x[0:4],x[5:9],x[10:14]
s=list()
def next_batch(x,step,batch_size):
    return x[batch_size*step:batch_size*step+batch_size]
for i in range(0,4):
    print(type(next_batch(x,i,124)))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Distancias entre vectores

* [Similitud coseno](http://reference.wolfram.com/language/ref/CosineDistance.html):
$$
CosineDistance[\vec{u},\vec{v}] = \frac{\vec{u}\cdot\vec{v}}{\lVert \vec{u} 
\lVert \lVert \vec{v} \lVert}
$$

---

* [Distancia euclidiana](http://reference.wolfram.com/language/ref/EuclideanDistance.html):
$$
EuclidianDistance[\vec{u},\vec{v}] = \sqrt{ \sum_{k=1}^n (u_i-v_i)^2}
$$

In [13]:
def get_distance(vector, matrix, distance="cos"):
    tmp_dist = list(enumerate(((np.matmul(vector, matrix.T) /
                                    (np.linalg.norm(vector) *
                                     np.sqrt(np.einsum('ij,ij->i',
                                                 matrix, matrix,dtype=np.float64)))))))
    # Se ordena lista según distancias más cercanas.
    tmp_dist = sorted(tmp_dist, key=lambda dist: dist[1],reverse=True)
    distances = tmp_dist[:10]
    del tmp_dist
    # Retorna 10 vectores más cercanos.
    return distances

In [14]:
%%time
c = None

c= list(enumerate(((np.matmul(y[i], x.T) /
       (np.linalg.norm(y[0]) *
        np.sqrt(np.einsum('ij,ij->i',
                          x, x,dtype=np.float64)))))))


CPU times: user 7.67 ms, sys: 24 ms, total: 31.7 ms
Wall time: 12 ms


In [15]:
%%time
c = None
c = [(((np.dot(x[0],x[i]))/(np.linalg.norm(x[0])*np.linalg.norm(x[i]))))
                for i in range(x.shape[0])]

CPU times: user 35.4 ms, sys: 0 ns, total: 35.4 ms
Wall time: 36.4 ms


In [16]:
%%time
dist = get_distance(y[0],x)

CPU times: user 5.18 ms, sys: 4.47 ms, total: 9.65 ms
Wall time: 4.85 ms


In [17]:
x.shape,y.shape

((1500, 300), (6000, 300))

In [18]:
def get_distance1(matrix_A,matrix_B,metric='cosine'):
    distances=1-cdist(matrix_A,matrix_B,metric=metric)
    tmp_distances,tmp_vect = [], []
    for i in range(distances.shape[0]):
        tmp_vect = list(enumerate(distances[i]))
        tmp_vect = sorted(tmp_vect, key=lambda dist: dist[1],reverse=True)
        tmp_distances.append(tmp_vect[:10])
        tmp_vect.clear()
    del tmp_vect
    del distances
    return tmp_distances

In [19]:
%%time
r=get_distance1(y,x)
print(len(r))

6000
CPU times: user 7.27 s, sys: 69.3 ms, total: 7.34 s
Wall time: 7.37 s


In [95]:
r[10]

[(220, 0.8566932893400767),
 (1492, 0.8465342385071933),
 (95, 0.8451393539208696),
 (869, 0.8447176125219888),
 (1242, 0.8440201503960736),
 (448, 0.8438546658924613),
 (798, 0.8427887122100223),
 (248, 0.8411944653609964),
 (198, 0.8411016149077818),
 (296, 0.8403410066824685)]

In [92]:
%%time
f=[get_distance(y[i],x) for i in range(y.shape[0])]

CPU times: user 4min 24s, sys: 5min 27s, total: 9min 51s
Wall time: 2min 43s


In [96]:
f[10]

[(220, 0.8566932893400767),
 (1492, 0.8465342385071933),
 (95, 0.8451393539208696),
 (869, 0.8447176125219888),
 (1242, 0.8440201503960736),
 (448, 0.8438546658924613),
 (798, 0.8427887122100224),
 (248, 0.8411944653609964),
 (198, 0.8411016149077818),
 (296, 0.8403410066824685)]

In [203]:
1-d

array([[5055.68784052, 4981.81984362, 5080.85080066, 4982.59840825,
        5100.94373183, 4974.57378256, 5040.01030049, 5028.57787744,
        5021.48424228, 4852.7099363 , 4991.08102625, 4913.97089524,
        4935.53465338, 4938.60001221, 4934.02528843, 5085.37193424,
        4958.02130919, 5016.08861661, 4977.8968105 , 5102.06042339,
        5100.43953042, 5126.51663282, 5102.49778823, 5078.62707068,
        5037.0970183 , 5054.19207738, 5032.6975903 , 4955.78676866,
        5053.84317158, 5016.40908041, 5022.39184062, 5049.72471705,
        5021.51300466, 5145.08522349, 5042.54027664, 4980.19014189,
        5050.21565044, 4929.610672  , 5162.14673972, 4987.71316111,
        5137.88021065, 5187.8306616 , 5158.37874648, 5097.15905548,
        4930.9328011 , 5106.1378127 , 5101.62366582, 5317.43618851,
        4972.95412696, 5185.16462654, 5016.6320545 , 5067.05994752,
        5075.30294006, 5064.33964668, 5289.95490528, 4929.60683942,
        5090.23809507, 5102.6749708 , 5203.41580

In [64]:
dist[:10]

[(0, 0.8118531207546604), (1, 0.8086630486444993), (2, 0.7563115858713536)]

In [25]:
dist[:10]

[(31, 0.9995134642623069),
 (86, 0.9991546422855964),
 (61, 0.9991124362103639),
 (67, 0.9989838415850201),
 (24, 0.998787341887897),
 (54, 0.9987124868388599),
 (94, 0.9985934740692571),
 (90, 0.9985356708406751),
 (39, 0.9980368514099769),
 (18, 0.997886914114308)]

In [26]:
dist = get_distance(y[0],x)

In [27]:
dist[:10]

[(31, 0.9995134642623069),
 (86, 0.9991546422855964),
 (61, 0.9991124362103639),
 (67, 0.9989838415850201),
 (24, 0.998787341887897),
 (54, 0.9987124868388599),
 (94, 0.9985934740692571),
 (90, 0.9985356708406751),
 (39, 0.9980368514099769),
 (18, 0.997886914114308)]

In [28]:
dist.sort(key=lambda tup: tup[1])

In [29]:
dist[:10]

[(18, 0.997886914114308),
 (39, 0.9980368514099769),
 (90, 0.9985356708406751),
 (94, 0.9985934740692571),
 (54, 0.9987124868388599),
 (24, 0.998787341887897),
 (67, 0.9989838415850201),
 (61, 0.9991124362103639),
 (86, 0.9991546422855964),
 (31, 0.9995134642623069)]